# Clase 11

Objetivos

1. Implementar minFlux
2. Implementar SPOT

In [1]:
import cobra.test
import cplex
import pandas as pd
import numpy as np

# 1. minFLUX

La ventaja de minFlux sobre FBA es que se obtiene una solución única. Sin embargo, minFlux elimina ciclos que se han observado como activos. Por ejemplo, en *Escherichia coli*  elimina el ciclo del glyoxilato, aún cuando este se ha observado como activo. Una alternativa (que no alcanzaremos a ver en esta clase) es [MaxEnt](https://journals.plos.org/plosone/article/authors?id=10.1371/journal.pone.0243067).

In [2]:
import optlang
optlang.available_solvers

{'GUROBI': False,
 'GLPK': True,
 'MOSEK': False,
 'CPLEX': True,
 'COINOR_CBC': False,
 'SCIPY': False,
 'OSQP': False}

In [3]:
#1. Compute biomass flux using FBA
model = cobra.test.create_test_model("ecoli")
solution_fba = model.optimize()

sumFluxes = 0
for reaction in model.reactions:
    sumFluxes += solution_fba[reaction.id]**2
print(model.objective.value, sumFluxes)

0.9823718127269753 19424.35396547605


In [7]:
#2. Run minFlux

In [9]:
#2.1 Set v_biomass to the value found using FBA

model.reactions.BIOMASS_Ec_iJO1366_core_53p95M.bounds=(solution_fba["BIOMASS_Ec_iJO1366_core_53p95M"]
                                          ,solution_fba["BIOMASS_Ec_iJO1366_core_53p95M"])

In [14]:
#2.1 Add Euclidean objecive function

for i, reaction in enumerate(model.reactions):
    if i == 0:
        euclidean_expression  = model.reactions.get_by_id(reaction.id).flux_expression**2
    else:
        euclidean_expression += model.reactions.get_by_id(reaction.id).flux_expression**2
        
euclidean_objective = model.problem.Objective(euclidean_expression,direction="min")
model.objective = euclidean_objective        

In [12]:
#2.3 Compute new distribution of fluxes
solution_minFlux = model.optimize()

In [13]:
sumFluxes = 0
for reaction in model.reactions:
    sumFluxes += solution_minFlux[reaction.id]**2
print(solution_minFlux["BIOMASS_Ec_iJO1366_core_53p95M"], model.objective.value, sumFluxes)

0.9823718127269753 15712.349723139301 15712.349723134696


## Ejercicio 1

Predecir los flujos de *Azotobacter vineldii* usando minFlux

# 2. SPOT

SPOT también produce soluciones únicas y además no necesita un reacción de biomasa. Lo último es especialmente útil para celulas pluricelulares, en las cuales no siempre es claro que la maximización de biomasa sea una función objetivo apropiada.

Usemos como ejemplo los datos recopilados por et al [Min Kyung Kim et al](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0157101#sec021).

In [2]:
# Load fpkm data and model

#wget -O S2.xls 'https://journals.plos.org/plosone/article/file?type=supplementary&id=info:doi/10.1371/journal.pone.0157101.s002'

data=pd.read_excel("S2.xls",sheet_name="Transcriptomic data - Holm", index_col=0, header=0) 
model = cobra.test.create_test_model("ecoli")

In [3]:
# Define una función para cargar los fpkm de acuerdo a las gene_reaction_rules

def getG(grl,fpkm):
    grl=grl.replace("(","")
    grl=grl.replace(")","")
    g = 0
    for subrule in grl.split("or"):
        g_subrule = []
        for gene in subrule.split("and"):
            gene = gene.replace(" ","")
            if gene in fpkm:
                g_subrule.append( float(fpkm[gene]) )
        if len(g_subrule)>=1: 
            g += min(g_subrule)
    return(g)

In [4]:
# Create a dictionary with the fpkm of each gene

fpkm={}

for gene in model.genes:
    if gene.id  in data.index:
        fpkm[gene] = data["Ref"].loc[gene.id]
    #else:
    #    print(gene.id," not found")

In [5]:
# Create a dictionary with the fpkm for each reaction
g = {}
mean_g = np.mean(list(fpkm.values()) ) # useful when not a grl available

for reaction in model.reactions:
    if reaction.gene_reaction_rule == "":
        g[reaction.id] = mean_g    
    else:
        g[reaction.id] = getG(reaction.gene_reaction_rule,fpkm)

In [6]:
#1 Define objecive function
model = cobra.test.create_test_model("ecoli")

for i, reaction in enumerate(model.reactions):
    if i == 0:
        dotProduct_expression  = model.reactions.get_by_id(reaction.id).forward_variable*g[reaction.id]
        dotProduct_expression += model.reactions.get_by_id(reaction.id).reverse_variable*g[reaction.id]
    else:
        dotProduct_expression += model.reactions.get_by_id(reaction.id).forward_variable*g[reaction.id]
        dotProduct_expression += model.reactions.get_by_id(reaction.id).reverse_variable*g[reaction.id]

In [7]:
dotProduct_objective = model.problem.Objective(dotProduct_expression,direction="min")
model.objective = dotProduct_objective 

In [8]:
#2.3 Compute new distribution of fluxes
solution_SPOT = model.optimize()

In [9]:
solution_SPOT

,fluxes,reduced_costs
DM_4crsol_c,-0.0,-19.171979
DM_5drib_c,0.0,37.392158
DM_aacald_c,0.0,46.451258
DM_amob_c,-0.0,-19.171979
DM_mththf_c,-0.0,-19.171979
...,...,...
ZN2abcpp,0.0,15.500749
ZN2t3pp,0.0,3.875187
ZN2tpp,0.0,0.000000
ZNabcpp,0.0,15.500749
